In [1]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [2]:
import numpy as np
import pathlib
import struct
import pandas as pd
import gc
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tabulate import tabulate
import csv

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

print("Done")

Done


In [3]:
### Usable Files

direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Filtered/Processed/")

# direction = pathlib.Path("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Detector97/Unfiltered/Processed/")

file_list = list(direction.iterdir())

file_num = len(file_list)

step = 0.8

num_test_files = 2
eighty = round(step*file_num)
twenty = file_num-eighty - num_test_files

list_of_file_ids_train = np.arange(eighty, dtype=int)
print(list_of_file_ids_train)
list_of_file_ids_val = np.arange(eighty,eighty+twenty-num_test_files, dtype=int)
print(list_of_file_ids_val)
list_of_file_ids_test =np.arange(file_num-num_test_files,file_num)
print(list_of_file_ids_test)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
[32 33 34 35]
[38 39]


In [4]:
##### Functions to process the data

        ### Unnormalization of each signal individually
def Unnormalized(batch_signals):
        
        return batch_signals
        
        ### Normalization of each signal individually
def Normalized(batch_signals):

        for i in range(len(batch_signals)):
            batch_signals[i] = batch_signals[i]/np.max(batch_signals[i])
            
        return batch_signals
            
        
        ### Normalization of the entire value by one common denominator      
def Denominator(batch_signals):  
    
        denominator = 3953.48
        batch_signals = batch_signals/denominator
        
        return batch_signals


##### Class

class TrainDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Train Class')
        if(file_id == 0):
#             print("reshuffling")
            np.random.shuffle(list_of_file_ids_train)             

        i_file = list_of_file_ids_train[file_id]

#         print(f'file_id: {file_id}, i_file: {i_file}')
#         print()
        signal_filename = direction/f'{i_file+1}.h5'

        
         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)  

        mask = (df['A'] == mass) & (df['Z'] == energy)
        df_new = df[~mask]       

        
        labels1 = df_new.iloc[:,9].values
        labels2 = df_new['ToF'].values
        labels = labels1+labels2
        
        signals = df_new[df_new.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels 
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )
    
class ValDataset(tf.data.Dataset):

    def _generator(file_id):  
#         print(f'Using Val Class')
        i_file = list_of_file_ids_val[file_id]
    
        signal_filename = direction/f'{i_file+1}.h5'

         # Load the labels and signals from the files
        df = pd.read_hdf(signal_filename,key=None)  

        mask = (df['A'] == mass) & (df['Z'] == energy)
        df_new = df[~mask]       

        
        labels1 = df_new.iloc[:,9].values
        labels2 = df_new['ToF'].values
        labels = labels1+labels2
        
        signals = df_new[df_new.columns[10:-2]].values
        
        
        # Determine how many batches can be made from this file
        num_batches = len(signals) // batch_size

        # Shuffle the signals within the file
        signal_indices = np.arange(len(signals))
        np.random.shuffle(signal_indices)        
        
        # Loop through each batch in the file
        for batch_idx in range(num_batches):
            # Get the signals and labels for this batch
            batch_signal_indices = signal_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]      
 
            batch_signals = signals[batch_signal_indices]
            
            batch_signals = Processing[process](batch_signals)
                
            batch_signals = batch_signals[:,:,np.newaxis] # Can also be done with signals = signals[:,:,np.newaxis]
            batch_labels = labels[batch_signal_indices]

            # Yield the signals and labels as a tuple
            yield batch_signals, batch_labels
             
    def __new__(cls, file_id):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float64, tf.dtypes.float64),
            output_shapes=((batch_size, 1998,1), (batch_size, )),
            args=(file_id,)
        )

In [5]:
def create_model():
    model = keras.models.Sequential()
    model.add(Conv1D(filters=16, kernel_size=5, activation='relu', input_shape=(1998, 1)))
    model.add(Conv1D(filters=8, kernel_size=5, dilation_rate=2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=4, kernel_size=5, dilation_rate=2, activation='relu'))
    model.add(Conv1D(filters=4, kernel_size=5, strides=2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=4, kernel_size=3, strides=2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    return model

In [6]:
# Pre-processing options
Processing = {
    "Unnormalized": Unnormalized,
    "Normalized": Normalized,
    "Denominator": Denominator
}
Process = ["Unnormalized","Normalized","Denominator"]
process = Process[2]
# Loss Function

loss_function = ['mean_absolute_error','mean_squared_error']
lf = 1

# Training Variables
batch_size = 32
num_epochs = 30

steps_per_epoch = eighty*5000 // batch_size

# Learning Rate
initial_lr = 1e-03
final_lr = 1e-06

# initial_lr = 1e-03
# final_lr = 1e-03

def step_decay(epoch):
    lrate = initial_lr * (final_lr/initial_lr)**(epoch/num_epochs)

    print(f'Current Learning rate: {lrate}')
    return lrate

# Compile the model
# model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

# Configuring training dataset
dataset_train = tf.data.Dataset.range(eighty).interleave(
        TrainDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).repeat().prefetch(1)



# Configuring training dataset
dataset_val = tf.data.Dataset.range(twenty-num_test_files).interleave(
        ValDataset,
        cycle_length=2,
        num_parallel_calls=2,
        deterministic=True).prefetch(1)


# Callback Functions
LRS = tf.keras.callbacks.LearningRateScheduler(step_decay)

ES = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True,verbose=1)

CSV = tf.keras.callbacks.CSVLogger("C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Data/Log.csv",
                                separator=",", append=True)

MC_path = f"C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/model_checkpoint.h5"
MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
)

callbacks = [MC,LRS,CSV]


Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [7]:
import pandas as pd

df = pd.read_hdf(file_list[0], key=None)

unique_mass = df['A'].unique()
# Assuming you have already loaded the DataFrame 'df'

# Step 1: Group by 'A' (mass) and 'Z' (energy) and count the occurrences
grouped = df.groupby(['A', 'Z']).size().reset_index(name='count')

mass_subgroups = grouped.groupby('A')

count = mass_subgroups.count

print(unique_mass)

# subgroup = mass_subgroups.get_group(85)
# for i in range(len(subgroup)):
#     print(subgroup.iloc[i][0])
#     print(subgroup.iloc[i][1])
#     mask = (df['A'] == subgroup.iloc[i][0]) & (df['Z'] == subgroup.iloc[i][1])
#     df_new = df[~mask]
#     print(len(df[mask]))

#     print(len(df_new))
mass_list = sorted(unique_mass)
print(mass_list)
for Mass in mass_list:
    
    subgroup = mass_subgroups.get_group(Mass)
    print()
    for i in range(len(subgroup)):
        mass = subgroup.iloc[i][0]
        energy = subgroup.iloc[i][1]

        print(mass,energy)
        



[100. 104. 105. 110. 127. 130. 133. 135. 136. 140. 143. 149.  80.  85.
  90.  91.  95.  97.]
[80.0, 85.0, 90.0, 91.0, 95.0, 97.0, 100.0, 104.0, 105.0, 110.0, 127.0, 130.0, 133.0, 135.0, 136.0, 140.0, 143.0, 149.0]

80.0 94.0

85.0 49.0
85.0 58.0
85.0 66.0
85.0 75.0
85.0 83.0
85.0 92.0
85.0 100.0

90.0 52.0
90.0 61.0
90.0 70.0
90.0 79.0
90.0 88.0
90.0 97.0
90.0 106.0

91.0 20.0
91.0 26.0
91.0 33.0
91.0 40.0
91.0 47.0
91.0 54.0

95.0 55.0
95.0 65.0
95.0 74.0
95.0 84.0
95.0 93.0
95.0 103.0

97.0 27.0
97.0 34.0
97.0 41.0
97.0 48.0
97.0 55.0

100.0 34.0
100.0 41.0
100.0 48.0
100.0 58.0
100.0 68.0
100.0 78.0
100.0 88.0
100.0 98.0
100.0 108.0

104.0 22.0
104.0 30.0
104.0 38.0
104.0 46.0
104.0 54.0
104.0 62.0

105.0 61.0
105.0 72.0
105.0 82.0
105.0 93.0
105.0 103.0

110.0 47.0
110.0 63.0
110.0 64.0
110.0 75.0
110.0 86.0
110.0 97.0

127.0 52.0
127.0 61.0

130.0 28.0
130.0 38.0
130.0 48.0
130.0 58.0
130.0 68.0
130.0 76.0
130.0 78.0
130.0 89.0

133.0 55.0
133.0 65.0

135.0 79.0

136.0 38.0
136.0 

In [8]:
num_runs = 3  # Number of times to run the model

df = pd.read_hdf(file_list[0], key=None)

unique_mass = df['A'].unique()
# Assuming you have already loaded the DataFrame 'df'

# Step 1: Group by 'A' (mass) and 'Z' (energy) and count the occurrences
grouped = df.groupby(['A', 'Z']).size().reset_index(name='count')

mass_subgroups = grouped.groupby('A')

#  133. 140. 149.


# 90.0 88.0
# 90.0 97.0
# 90.0 106.0


mass = 90.0
energy = 88.0

print(mass,energy)

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")

    model = create_model()

    model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

    MC_path = f"C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/model_{mass}_{energy}_checkpoint_{run + 1}.h5"
    MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
    )

    callbacks = [MC,LRS,CSV]

    # Train the model
    history = model.fit(x=dataset_train, validation_data = dataset_val, steps_per_epoch=steps_per_epoch, epochs=num_epochs,callbacks=callbacks)        

    
    
mass = 90.0
energy = 97.0

print(mass,energy)

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")

    model = create_model()

    model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

    MC_path = f"C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/model_{mass}_{energy}_checkpoint_{run + 1}.h5"
    MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
    )

    callbacks = [MC,LRS,CSV]

    # Train the model
    history = model.fit(x=dataset_train, validation_data = dataset_val, steps_per_epoch=steps_per_epoch, epochs=num_epochs,callbacks=callbacks)        
    
    
mass = 90.0
energy = 106.0

print(mass,energy)

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")

    model = create_model()

    model.compile(loss=loss_function[lf], optimizer = keras.optimizers.Adam(initial_lr), metrics=['mean_absolute_error','mean_squared_error'])

    MC_path = f"C:/Users/steve/OneDrive/Bureaublad/Studies/Thesis/Trainings/model_{mass}_{energy}_checkpoint_{run + 1}.h5"
    MC = ModelCheckpoint(
    filepath=MC_path,  # Filepath to save the model weights
    monitor='val_loss',  # Quantity to monitor (e.g., validation loss)
    save_best_only=True,  # Save only the best model based on the monitored quantity
    save_weights_only=True  # Save only the model weights, not the entire model
    )

    callbacks = [MC,LRS,CSV]

    # Train the model
    history = model.fit(x=dataset_train, validation_data = dataset_val, steps_per_epoch=steps_per_epoch, epochs=num_epochs,callbacks=callbacks)        
    
    
    

                 

90.0 88.0
Run 1/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 99s 14ms/step - loss: 1298.8530 - mean_absolute_error: 4.0536 - mean_squared_error: 1298.8530 - val_loss: 0.1926 - val_mean_absolute_error: 0.2879 - val_mean_squared_error: 0.1926
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 59s 12ms/step - loss: 0.3634 - mean_absolute_error: 0.4726 - mean_squared_error: 0.3634 - val_loss: 2.7382 - val_mean_absolute_error: 1.6483 - val_mean_squared_error: 2.7382
Epoch 3/30
Current Learning rate: 0.0006309573444801933
5000/5000 [==============================] - 59s 12ms/step - loss: 0.2739 - mean_absolute_error: 0.4171 - mean_squared_error: 0.2739 - val_loss: 0.5208 - val_mean_absolute_error: 0.7100 - val_mean_squared_error: 0.5208
Epoch 4/30
Current Learning rate: 0.0005011872336272722
5000/5000 [==============================] - 58s 12ms/step - loss: 0.1409 - mean_absolute_error: 0.2955 - mean_

5000/5000 [==============================] - 58s 12ms/step - loss: 0.0116 - mean_absolute_error: 0.0797 - mean_squared_error: 0.0116 - val_loss: 0.0111 - val_mean_absolute_error: 0.0778 - val_mean_squared_error: 0.0111
Run 2/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 72s 13ms/step - loss: 1659.9707 - mean_absolute_error: 4.8607 - mean_squared_error: 1659.9707 - val_loss: 8.3965 - val_mean_absolute_error: 2.8876 - val_mean_squared_error: 8.3965
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 56s 11ms/step - loss: 0.2120 - mean_absolute_error: 0.3483 - mean_squared_error: 0.2120 - val_loss: 103.7178 - val_mean_absolute_error: 10.1819 - val_mean_squared_error: 103.7178
Epoch 3/30
Current Learning rate: 0.0006309573444801933
5000/5000 [==============================] - 66s 13ms/step - loss: 0.1559 - mean_absolute_error: 0.3126 - mean_squared_error: 0.1559 - val_loss: 2.1079 - val_mean_absolute_

Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 66s 13ms/step - loss: 0.0116 - mean_absolute_error: 0.0788 - mean_squared_error: 0.0116 - val_loss: 0.0107 - val_mean_absolute_error: 0.0750 - val_mean_squared_error: 0.0107
Run 3/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 71s 12ms/step - loss: 1294.1368 - mean_absolute_error: 4.0440 - mean_squared_error: 1294.1368 - val_loss: 0.4775 - val_mean_absolute_error: 0.6507 - val_mean_squared_error: 0.4775
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 60s 12ms/step - loss: 0.5145 - mean_absolute_error: 0.5629 - mean_squared_error: 0.5145 - val_loss: 0.0479 - val_mean_absolute_error: 0.1809 - val_mean_squared_error: 0.0479
Epoch 3/30
Current Learning rate: 0.0006309573444801933
5000/5000 [==============================] - 59s 12ms/step - loss: 0.3330 - mean_absolute_error: 0.4512 - mean_squared_

Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 67s 13ms/step - loss: 0.0139 - mean_absolute_error: 0.0895 - mean_squared_error: 0.0139 - val_loss: 0.0103 - val_mean_absolute_error: 0.0743 - val_mean_squared_error: 0.0103
90.0 97.0
Run 1/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 68s 12ms/step - loss: 1538.9492 - mean_absolute_error: 4.4555 - mean_squared_error: 1538.9492 - val_loss: 9.6892 - val_mean_absolute_error: 3.0965 - val_mean_squared_error: 9.6892
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 50s 10ms/step - loss: 8.3401 - mean_absolute_error: 0.7191 - mean_squared_error: 8.3401 - val_loss: 3.0774 - val_mean_absolute_error: 1.6312 - val_mean_squared_error: 3.0774
Epoch 3/30
Current Learning rate: 0.0006309573444801933
5000/5000 [==============================] - 63s 13ms/step - loss: 0.7969 - mean_absolute_error: 0.6321 - mea

5000/5000 [==============================] - 63s 13ms/step - loss: 0.0126 - mean_absolute_error: 0.0822 - mean_squared_error: 0.0126 - val_loss: 0.0146 - val_mean_absolute_error: 0.0900 - val_mean_squared_error: 0.0146
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 50s 10ms/step - loss: 0.0128 - mean_absolute_error: 0.0821 - mean_squared_error: 0.0128 - val_loss: 0.0118 - val_mean_absolute_error: 0.0792 - val_mean_squared_error: 0.0118
Run 2/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 73s 12ms/step - loss: 1503.0846 - mean_absolute_error: 5.7991 - mean_squared_error: 1503.0846 - val_loss: 1.9130 - val_mean_absolute_error: 1.3193 - val_mean_squared_error: 1.9130
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 47s 9ms/step - loss: 0.4025 - mean_absolute_error: 0.4865 - mean_squared_error: 0.4025 - val_loss: 0.6565 - val_mean_absolute_erro

Epoch 29/30
Current Learning rate: 1.5848931924611134e-06
5000/5000 [==============================] - 67s 13ms/step - loss: 0.0125 - mean_absolute_error: 0.0825 - mean_squared_error: 0.0125 - val_loss: 0.0207 - val_mean_absolute_error: 0.1156 - val_mean_squared_error: 0.0207
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 48s 10ms/step - loss: 0.0123 - mean_absolute_error: 0.0821 - mean_squared_error: 0.0123 - val_loss: 0.0332 - val_mean_absolute_error: 0.1560 - val_mean_squared_error: 0.0332
Run 3/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 68s 12ms/step - loss: 1596.2788 - mean_absolute_error: 4.3856 - mean_squared_error: 1596.2788 - val_loss: 0.5515 - val_mean_absolute_error: 0.6759 - val_mean_squared_error: 0.5515
Epoch 2/30
Current Learning rate: 0.0007943282347242815
5000/5000 [==============================] - 48s 10ms/step - loss: 0.4659 - mean_absolute_error: 0.4863 - mean_square

5000/5000 [==============================] - 48s 10ms/step - loss: 0.0108 - mean_absolute_error: 0.0764 - mean_squared_error: 0.0108 - val_loss: 0.0106 - val_mean_absolute_error: 0.0760 - val_mean_squared_error: 0.0106
Epoch 29/30
Current Learning rate: 1.5848931924611134e-06
5000/5000 [==============================] - 63s 13ms/step - loss: 0.0107 - mean_absolute_error: 0.0761 - mean_squared_error: 0.0107 - val_loss: 0.0106 - val_mean_absolute_error: 0.0756 - val_mean_squared_error: 0.0106
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 48s 10ms/step - loss: 0.0107 - mean_absolute_error: 0.0762 - mean_squared_error: 0.0107 - val_loss: 0.0108 - val_mean_absolute_error: 0.0760 - val_mean_squared_error: 0.0108
90.0 106.0
Run 1/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 64s 11ms/step - loss: 2358.2131 - mean_absolute_error: 6.3971 - mean_squared_error: 2358.2131 - val_loss: 0.2689 - val_mean

Epoch 28/30
Current Learning rate: 1.9952623149688796e-06
5000/5000 [==============================] - 49s 10ms/step - loss: 0.0123 - mean_absolute_error: 0.0826 - mean_squared_error: 0.0123 - val_loss: 0.0112 - val_mean_absolute_error: 0.0779 - val_mean_squared_error: 0.0112
Epoch 29/30
Current Learning rate: 1.5848931924611134e-06
5000/5000 [==============================] - 60s 12ms/step - loss: 0.0121 - mean_absolute_error: 0.0822 - mean_squared_error: 0.0121 - val_loss: 0.0111 - val_mean_absolute_error: 0.0776 - val_mean_squared_error: 0.0111
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 50s 10ms/step - loss: 0.0122 - mean_absolute_error: 0.0820 - mean_squared_error: 0.0122 - val_loss: 0.0114 - val_mean_absolute_error: 0.0786 - val_mean_squared_error: 0.0114
Run 2/3
Epoch 1/30
Current Learning rate: 0.001
5000/5000 [==============================] - 65s 11ms/step - loss: 2259.7034 - mean_absolute_error: 5.9770 - mean_squared

5000/5000 [==============================] - 65s 13ms/step - loss: 0.0143 - mean_absolute_error: 0.0909 - mean_squared_error: 0.0143 - val_loss: 0.0173 - val_mean_absolute_error: 0.1014 - val_mean_squared_error: 0.0173
Epoch 28/30
Current Learning rate: 1.9952623149688796e-06
5000/5000 [==============================] - 49s 10ms/step - loss: 0.0144 - mean_absolute_error: 0.0907 - mean_squared_error: 0.0144 - val_loss: 0.0192 - val_mean_absolute_error: 0.1080 - val_mean_squared_error: 0.0192
Epoch 29/30
Current Learning rate: 1.5848931924611134e-06
5000/5000 [==============================] - 64s 13ms/step - loss: 0.0142 - mean_absolute_error: 0.0903 - mean_squared_error: 0.0142 - val_loss: 0.0178 - val_mean_absolute_error: 0.1036 - val_mean_squared_error: 0.0178
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 48s 10ms/step - loss: 0.0144 - mean_absolute_error: 0.0907 - mean_squared_error: 0.0144 - val_loss: 0.0173 - val_mean_absolu

Epoch 27/30
Current Learning rate: 2.5118864315095793e-06
5000/5000 [==============================] - 62s 12ms/step - loss: 0.0120 - mean_absolute_error: 0.0802 - mean_squared_error: 0.0120 - val_loss: 0.0129 - val_mean_absolute_error: 0.0854 - val_mean_squared_error: 0.0129
Epoch 28/30
Current Learning rate: 1.9952623149688796e-06
5000/5000 [==============================] - 53s 11ms/step - loss: 0.0120 - mean_absolute_error: 0.0797 - mean_squared_error: 0.0120 - val_loss: 0.0108 - val_mean_absolute_error: 0.0752 - val_mean_squared_error: 0.0108
Epoch 29/30
Current Learning rate: 1.5848931924611134e-06
5000/5000 [==============================] - 63s 13ms/step - loss: 0.0117 - mean_absolute_error: 0.0792 - mean_squared_error: 0.0117 - val_loss: 0.0110 - val_mean_absolute_error: 0.0761 - val_mean_squared_error: 0.0110
Epoch 30/30
Current Learning rate: 1.2589254117941674e-06
5000/5000 [==============================] - 49s 10ms/step - loss: 0.0118 - mean_absolute_error: 0.0789 - mean_

In [9]:
import pandas as pd

df = pd.read_hdf(file_list[0], key=None)
df['A'] = pd.to_numeric(df['A'])
df['Z'] = pd.to_numeric(df['Z'])

unique_mass = df['A'].unique()

hope = []

for mass_u in unique_mass:
    df_intermediate = df[df['A'] == mass_u]
    unique_energy = df_intermediate['Z'].unique()

    for energy_u in unique_energy:
        df_copy = df.copy()  # Create a copy of the original DataFrame

        print(f"Size of original panda file: {len(df_copy)}")
        print(mass_u, energy_u) 

        # Step 1: Apply the first condition and create an intermediate DataFrame
        df_intermediate = df_copy[df_copy['A'] == mass_u]

        # Step 2: Apply the second condition to the intermediate DataFrame and create the final filtered DataFrame
        df_filtered = df_intermediate[df_intermediate['Z'] == energy_u]

        # Check for duplicates in df_copy
        duplicated_rows = df_copy[df_copy.duplicated()]
#         print(f"Duplicated rows in df_copy:\n{duplicated_rows}")

        # Get the indices of rows to be dropped from df_copy
        rows_to_drop = df_filtered.index

        # Drop the rows from df_copy
        df_copy = df_copy.drop(rows_to_drop)

        # Drop all duplicate rows
        df_copy = df_copy.drop_duplicates()
        
        print(f'Number of deleted rows: {len(df_filtered)}')
#         print(f'Size of new panda file: {len(df_copy)}')
        print()

Size of original panda file: 10055
100.0 108
Number of deleted rows: 17

Size of original panda file: 10055
100.0 34
Number of deleted rows: 104

Size of original panda file: 10055
100.0 41
Number of deleted rows: 94

Size of original panda file: 10055
100.0 48
Number of deleted rows: 73

Size of original panda file: 10055
100.0 58
Number of deleted rows: 303

Size of original panda file: 10055
100.0 68
Number of deleted rows: 390

Size of original panda file: 10055
100.0 78
Number of deleted rows: 302

Size of original panda file: 10055
100.0 88
Number of deleted rows: 85

Size of original panda file: 10055
100.0 98
Number of deleted rows: 170

Size of original panda file: 10055
104.0 22
Number of deleted rows: 59

Size of original panda file: 10055
104.0 30
Number of deleted rows: 177

Size of original panda file: 10055
104.0 38
Number of deleted rows: 206

Size of original panda file: 10055
104.0 46
Number of deleted rows: 75

Size of original panda file: 10055
104.0 54
Number of de

In [10]:
import pandas as pd

df = pd.read_hdf(file_list[0], key=None)
df['A'] = pd.to_numeric(df['A'])
df['Z'] = pd.to_numeric(df['Z'])

unique_mass = df['A'].unique()

mass_counts = {}

for mass_u in unique_mass:
    df_intermediate = df[df['A'] == mass_u]
    unique_energy = df_intermediate['Z'].unique()

    energy_count = len(unique_energy)  # Number of energies for the current Mass (A)

    for energy_u in unique_energy:
        df_copy = df.copy()  # Create a copy of the original DataFrame

        # Step 1: Apply the first condition and create an intermediate DataFrame
        df_intermediate = df_copy[df_copy['A'] == mass_u]

        # Step 2: Apply the second condition to the intermediate DataFrame and create the final filtered DataFrame
        df_filtered = df_intermediate[df_intermediate['Z'] == energy_u]

        # Get the indices of rows to be dropped from df_copy
        rows_to_drop = df_filtered.index

        # Drop the rows from df_copy
        df_copy = df_copy.drop(rows_to_drop)

        # Drop all duplicate rows
        df_copy = df_copy.drop_duplicates()

        # Calculate the count for the current Mass (A)
        count = len(df_filtered)

        # Store the count in the dictionary
        if mass_u in mass_counts:
            mass_counts[mass_u].append(count)
        else:
            mass_counts[mass_u] = [count]

# Calculate the average count for each Mass (A)
average_counts = {mass: sum(counts) / len(counts) for mass, counts in mass_counts.items()}

print("Number of energies for each Mass (A):")
for mass, energy_counts in mass_counts.items():
    print(f"Mass {mass}: {len(energy_counts)} energies, Average Count: {average_counts[mass]}")

# Find the Mass (A) with the highest average count
max_average_mass = max(average_counts, key=average_counts.get)

print()
print(f"The Mass with the highest average count is {max_average_mass} with an average count of {average_counts[max_average_mass]}")

Number of energies for each Mass (A):
Mass 100.0: 9 energies, Average Count: 170.88888888888889
Mass 104.0: 6 energies, Average Count: 102.5
Mass 105.0: 5 energies, Average Count: 223.6
Mass 110.0: 6 energies, Average Count: 31.333333333333332
Mass 127.0: 2 energies, Average Count: 44.5
Mass 130.0: 8 energies, Average Count: 139.875
Mass 133.0: 2 energies, Average Count: 102.0
Mass 135.0: 1 energies, Average Count: 197.0
Mass 136.0: 5 energies, Average Count: 107.0
Mass 140.0: 3 energies, Average Count: 321.6666666666667
Mass 143.0: 4 energies, Average Count: 442.75
Mass 149.0: 2 energies, Average Count: 34.0
Mass 80.0: 1 energies, Average Count: 1.0
Mass 85.0: 7 energies, Average Count: 12.857142857142858
Mass 90.0: 7 energies, Average Count: 51.142857142857146
Mass 91.0: 6 energies, Average Count: 32.333333333333336
Mass 95.0: 6 energies, Average Count: 158.83333333333334
Mass 97.0: 5 energies, Average Count: 10.4

The Mass with the highest average count is 143.0 with an average coun